# 数据包采集与标注工具设计

本工具结合 Scapy、Wireshark 和 Label Studio，实现网络数据包的采集、分析与标注：

1. **采集**：使用 Scapy 进行网络数据包的实时采集，并保存为 pcap 文件。
2. **分析**：可用 Wireshark 对 pcap 文件进行可视化分析，辅助理解数据内容。
3. **标注**：将数据包基本信息导出为 CSV/JSON，通过 Label Studio 进行批量标注，适用于机器学习等场景。

整体流程如下：
- Scapy 采集数据包 → 保存为 pcap → 提取关键信息 → 导出为 CSV/JSON → Label Studio 标注

In [1]:
from kamene.all import *

In [ ]:
# 采集数据包并保存为 pcap 文件
def capture_and_save(count=20, iface=None, filename='packets.pcap'):
    packets = sniff(count=count, iface=iface)
    wrpcap(filename, packets)
    print(f'已保存 {count} 个数据包到 {filename}')

# 示例：采集10个数据包并保存
# capture_and_save(count=10, filename='sample.pcap')

In [ ]:
# 从 pcap 文件中提取关键信息并导出为 CSV，便于 Label Studio 标注
import pandas as pd

def extract_packet_info(pcap_file, csv_file):
    packets = rdpcap(pcap_file)
    data = []
    for pkt in packets:
        info = {
            'time': pkt.time,
            'src': pkt[IP].src if IP in pkt else '',
            'dst': pkt[IP].dst if IP in pkt else '',
            'proto': pkt[IP].proto if IP in pkt else '',
            'len': len(pkt)
        }
        data.append(info)
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False)
    print(f'已导出数据包信息到 {csv_file}')

# 示例：extract_packet_info('sample.pcap', 'packets.csv')

## 标注流程说明

1. 在 Label Studio 创建新项目，导入导出的 CSV 文件。
2. 配置标注界面（如协议类型、用途等标签）。
3. 进行人工标注，导出标注结果用于后续分析或模型训练。

In [ ]:
# ...existing code...